In [ ]:
!pip install openai-agents

In [ ]:
import os
from google.colab import userdata

# Ensure OPENAI_API_KEY is added to secrets
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

In [ ]:
from agents import Agent, Runner, ModelSettings, trace, gen_trace_id
from agents import WebSearchTool, CodeInterpreterTool

In [ ]:
print("OpenAI Agents SDK version:", __import__('agents').__version__)

OpenAI Agents SDK version: 0.1.0


In [ ]:
from pydantic import BaseModel
# Define structured output models for clarity
class ShoppingItem(BaseModel):
    name: str
    price: float
    url: str

class ShoppingList(BaseModel):
    items: list[ShoppingItem]

class TotalCostSummary(BaseModel):
    total: float
    item_count: int
    details: str

# Define agents
winter_agent = Agent(
    name="Winter Shopping List Agent",
    instructions="""
    You are an agent that generates a random shopping list for a complete  winter outfit.
    Include various items like a jacket, scarf, gloves, hat, boots, etc.  For each item, provide a name, a realistic price, and a placeholder URL.  Return the list as a ShoppingList.
    """,
    output_type=ShoppingList
)

cost_agent = Agent(
    name="Total Cost Agent",
    instructions="""
    You are a deterministic agent. Given a ShoppingList, calculate the total  cost and item count,
    and return a TotalCostSummary with a details string in the format:  'Total for X items: $Y'.
    """,
    output_type=TotalCostSummary
)

# Execute sequentially (deterministic flow):
shopping_list_result = await Runner.run(winter_agent, input="")
shopping_list = shopping_list_result.final_output

cost_result = await Runner.run(cost_agent,
                                input=shopping_list.model_dump_json())
summary = cost_result.final_output

print("Summary:", summary.details)

Summary: Total for 5 items: $350.00


Let's create some test cases for the agents. First, we'll test the `winter_agent` to see if it generates a valid `ShoppingList`.

In [ ]:
# Test case for winter_agent
winter_agent_test_result = await Runner.run(winter_agent, input="")
winter_agent_output = winter_agent_test_result.final_output

print("Winter Agent Output Type:", type(winter_agent_output))
print("Winter Agent Output (first 3 items):", winter_agent_output.items[:3])

Winter Agent Output Type: <class '__main__.ShoppingList'>
Winter Agent Output (first 3 items): [ShoppingItem(name='Insulated Winter Jacket', price=120.99, url='http://example.com/insulated-winter-jacket'), ShoppingItem(name='Wool Knit Scarf', price=25.0, url='http://example.com/wool-knit-scarf'), ShoppingItem(name='Leather Winter Gloves', price=35.5, url='http://example.com/leather-winter-gloves')]


Now, let's test the `cost_agent` with a sample `ShoppingList` to verify the total cost calculation.

In [ ]:
# Test case for cost_agent
sample_shopping_list = ShoppingList(items=[
    ShoppingItem(name="Hat", price=20.00, url="placeholder"),
    ShoppingItem(name="Gloves", price=25.00, url="placeholder"),
    ShoppingItem(name="Scarf", price=30.00, url="placeholder"),
])

cost_agent_test_result = await Runner.run(cost_agent,
                                          input=sample_shopping_list.model_dump_json())
cost_agent_output = cost_agent_test_result.final_output

print("Cost Agent Output Type:", type(cost_agent_output))
print("Cost Agent Output:", cost_agent_output)

Cost Agent Output Type: <class '__main__.TotalCostSummary'>
Cost Agent Output: total=75.0 item_count=3 details='Total for 3 items: $75'
